In [1]:
timespan = '66months'
rollingAverage = '1hour'
start = '2012-04-00'
sampled = 'false'

global totalled_df
global url
url = 'https://api.blockchain.info/charts/'

In [2]:
def create_df(url, chartName):
    url += chartName
    params = dict(
        timespan = timespan,
        rollingAverage = rollingAverage,
        start = start, 
        sampled = sampled,
    )
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
#     print(json.dumps(data, indent=4, sort_keys=True))
    data = data['values']
    df = pd.DataFrame(data)
    df.set_index('x', inplace=True)
    df.columns =[chartName]
    return df

In [3]:
def concat_df(all_df, new_df):
    all_df = pd.concat([all_df, new_df], axis=1, join_axes=[all_df.index])
#     totalled_df.T.drop_duplicates().T    
    all_df = all_df.loc[:,~all_df.columns.duplicated()]
    return all_df

***MARKET INFO***

In [4]:
import json, requests
import pandas as pd

#1 BTC MARKET PRICE
df = create_df(url, 'market-price')

totalled_df = df

In [5]:
#2 TRADE VOLUME
df = create_df(url, 'trade-volume')

totalled_df = concat_df(totalled_df, df)

***MINING INFORMATION***

In [6]:
#3 MEDIAN TRANSACTION TIME
df = create_df(url, 'median-confirmation-time')

totalled_df = concat_df(totalled_df, df)

In [7]:
#4 HASH RATE
df = create_df(url, 'hash-rate')

totalled_df = concat_df(totalled_df, df)

In [8]:
#5 TOTAL TRANSACTION FEES
df = create_df(url, 'transaction-fees-usd')

totalled_df = concat_df(totalled_df, df)

***NETWORK INFORMATION***

In [9]:
#6 NUMBER OF UNIQUE ADDRESSES
df = create_df(url, 'n-unique-addresses')

totalled_df = concat_df(totalled_df, df)

In [10]:
#7 TOTAL NUMBER OF TRANSACTIONS PER DAY
df = create_df(url, 'n-transactions')

totalled_df = concat_df(totalled_df, df)

In [11]:
#8 NUMBER OF TRANSACTIONS WAITING TO BE INCLUDED (MEMPOOL TRANSACTION COUNT)
df = create_df(url, 'mempool-count')

totalled_df = concat_df(totalled_df, df)

In [12]:
#9 TRANSACTION VOLUME (BTC) EXCLUDING TOP 100 POPULAR ADDRESSES
df = create_df(url, 'n-transactions-excluding-popular')

totalled_df = concat_df(totalled_df, df)

In [13]:
totalled_df.head()

,market-price,trade-volume,median-confirmation-time,hash-rate,transaction-fees-usd,n-unique-addresses,n-transactions,mempool-count,n-transactions-excluding-popular
x,,,,,,,,,
1333238400,4.92,182041.75,8.74,12.12,21.40,9586,5619,NaN,4221
1333324800,5.08,429855.97,11.46,10.26,21.03,10766,6086,NaN,4676
1333411200,5.00,394194.22,10.03,10.34,23.17,13073,7539,NaN,5962
1333497600,4.96,281521.13,10.15,11.40,19.64,12774,7479,NaN,5980
1333584000,4.94,187559.84,10.27,9.62,30.71,11946,6977,NaN,5552


In [14]:
totalled_df.to_csv('totalled_df.csv', sep=',')